In [12]:
from truecolours_loader import load_for_binary_classification,load
import numpy as np
import pandas as pd

In [10]:
df = load_for_binary_classification()

Processing Patient : 100%|██████████| 2496/2496 [03:12<00:00, 12.95it/s]


Formatting the loaded dataframe
Formatting for binary classification task
	 Filter to patients with UCEIS and NANCY test results
	 Aligning dates on UCEIS and NANCY test results
Formatting into X,y format


In [49]:
ground_truth_events = df.loc[ ~df['Nancy Index'].isna() | ~df['UCEIS (0 to 8)'].isna(), ['participant_id', 'response_date']].drop_duplicates()
ground_truth_events['cutoff_date'] = ground_truth_events['response_date'] - pd.to_timedelta(f'{60} days')

In [50]:
X = []
y = []

In [51]:
for (idx,row) in ground_truth_events.iterrows():
    
    # Filter main df to just the dates and patient in question
    patient_id,truth_date,start_date = row.values
    local_df = df.loc[
                        (df['participant_id'] == patient_id) &
                        (df['response_date']  >= start_date) &
                        (df['response_date']  <= truth_date)
                     ]
    
    # Compute y
    UCEIS = local_df['UCEIS (0 to 8)'].mean(skipna=True)
    NANCY = local_df['Nancy Index'].mean(skipna=True)
    local_y = (UCEIS >= 3.0) & (NANCY >= 2.0) # Criterion for active disease
    
    # Select columns for X
    local_x = local_df[['summary', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6']]
    
    X.append(local_x)
    y.append(local_y)

In [52]:
X[1]

,summary,q1,q2,q3,q4,q5,q6
1695,16.00,0.0,0,0,0,0,0
1696,16.00,0.0,0,1,0,0,0
1697,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1698,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1699,16.00,0.0,0,1,0,0,0
...,...,...,...,...,...,...,...
1752,3.57,14.0,0,14,10,0,12
1753,3.40,12.0,0,10,7,0,14
1754,NaN,1.0,78,162,NaN,3,2
1755,NaN,NaN,NaN,NaN,NaN,NaN,NaN
